In [14]:
import gc
import pandas as pd
import numpy as np
import os
import lightgbm as lgb
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [15]:
from sklearn.metrics import roc_curve 
def score(y,pred): 
    fpr, tpr, thresholds = roc_curve(y, pred, pos_label=1) 
    score=0.4*tpr[np.where(fpr>=0.001)[0][0]]+0.3*tpr[np.where(fpr>=0.005)[0][0]]+0.3*tpr[np.where(fpr>=0.01)[0][0]] 
    return score 
# from datetime import date
# def getWeekday(row):
#     if row == 'null':
#         return row
#     else:
#         return date(int(row[0:4]), int(row[4:6]), int(row[6:8])).weekday() + 1

In [16]:
####################训练集#######################
DATA_PATH = "../"
csv_path = os.path.join(DATA_PATH, "atec_anti_fraud_train.csv")
data = pd.read_csv(csv_path)

In [17]:
data.loc[data['label'] == -1, 'label'] = 1
data.loc[data['label'] == 1, 'label'] = 2
data.loc[data['label'] == 0, 'label'] = 1
data.loc[data['label'] == 2, 'label'] = 0

In [18]:
####################测试集#######################
csv_path = os.path.join(DATA_PATH, "atec_anti_fraud_test_b.csv")
data_test = pd.read_csv(csv_path)

In [19]:
data.fillna(0, inplace=True)
data_test.fillna(0, inplace=True)

In [20]:
data_train = data[data['date'] <= 20171021]
data_val = data[data['date'] > 20171021]

print(data.shape)
print(data_train.shape)
print(data_val.shape)

(994731, 300)
(749690, 300)
(245041, 300)


In [21]:
X = data.loc[:,'f1':].as_matrix()
y = data.loc[:,'label'].as_matrix()

X_train = data_train.loc[:,'f1':].as_matrix()
y_train = data_train.loc[:,'label'].as_matrix()

X_val = data_val.loc[:,'f1':].as_matrix()
y_val = data_val.loc[:,'label'].as_matrix()

X_test = data_test.loc[:,'f1':].as_matrix()

In [22]:
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)
print(X.shape)
print(y.shape)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)

(994731, 297)
(994731,)
(749690, 297)
(749690,)
(245041, 297)
(245041,)
(500538, 297)


In [23]:
del data, data_train, data_val
gc.collect()

123

In [24]:
def eval_func(y_pred, y_true):
    fpr, tpr, thresholds = metrics.roc_curve(y_pred, y_true, pos_label=1)
    score = 0.4 * tpr[np.argwhere(fpr < 0.001)[-1]] + 0.3 * tpr[np.argwhere(
        fpr < 0.005)[-1]] + 0.3 * tpr[np.argwhere(fpr < 0.01)[-1]]
    return 'score', score, True

# def eval_func(y_true, y_pred):
#     y_pred = y_pred.get_label()
#     fpr, tpr, thresholds = metrics.roc_curve(y_pred, y_true, pos_label=1)
#     score = 0.4 * tpr[np.argwhere(fpr < 0.001)[-1]] + 0.3 * tpr[np.argwhere(
#         fpr < 0.005)[-1]] + 0.3 * tpr[np.argwhere(fpr < 0.01)[-1]]
#     return 'score', score, True

In [25]:
clf = GradientBoostingClassifier(
    n_estimators=30,
    subsample=0.8,
    max_depth=6,
    max_leaf_nodes=50,
    max_features=0.9,
    learning_rate=0.1,
    random_state=0,
    verbose=1
)
clf.fit(X, y)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.1216           0.0517           30.32m
         2           0.1150           0.0064           28.73m
         3           0.1096           0.0051           27.53m
         4           0.1052           0.0036           26.27m
         5           0.1022           0.0035           24.71m
         6           0.0989           0.0029           23.76m
         7           0.0964           0.0024           22.65m
         8           0.0942           0.0020           21.74m
         9           0.0921           0.0018           20.55m
        10           0.0905           0.0016           19.56m
        20           0.0808           0.0004            9.46m
        30           0.0756           0.0002            0.00s


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=6,
              max_features=0.9, max_leaf_nodes=50,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=30,
              presort='auto', random_state=0, subsample=0.8, verbose=1,
              warm_start=False)

In [26]:
score(1-y, clf.predict_proba(X)[:, 0])

0.46188638926811898

In [32]:
y_val_pred = clf.predict_proba(X_val)[:,1]

In [27]:
y_pred = clf.predict_proba(X_test)[:,0]
result = pd.DataFrame({'id':data_test['id'].as_matrix(), 'score':y_pred})
result.to_csv("submission.csv", index=False)